In [2]:
# Import dependencies
import datetime
import dateutil
import numpy as np 
from openclsim.appendix import get_event_log
import openclsim.model as model
import openclsim.plugins as plugins
import os
import pandas as pd
import simpy

In [4]:
def create_sample_data():
    """Create sample data for the test simulation."""
    # Generate sample dates
    dates = np.arange(
        start=datetime.datetime(2021, 1, 1),
        stop=datetime.datetime(2021, 1, 2),
        step=datetime.timedelta(hours=1),
    )

    # Generate sample data and export to *.csv
    hs_values = np.random.random(size=(len(dates)))
    tp_values = np.random.random(size=(len(dates)))

    hs_df = pd.DataFrame({"date": dates, "hs": hs_values}).set_index("date")
    tp_df = pd.DataFrame({"date": dates, "tp": tp_values}).set_index("date")

    hs_df.to_csv("../tests/data/unit_hs.csv")
    tp_df.to_csv("../tests/data/unit_tp.csv")

    return

In [6]:
# Create a simulation environment
class SimulationEnvironment(object):
    """
    A OpenCLSim simulation object.

    In this example, a basic activity is modelled using the
    `model.BasicActivity` class. However, the activity is constrained
    to operational limits and may, therefore, only start when the
    offshore conditions are safe. The offshore environment is modelled
    using the `plugins.OffshoreEnvironment` and the corresponding
    waiting on weather event/activity is modelled using the plugin:
    `plugins.HasRequestWindowPluginActivity`.
    
    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.
    """
    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)
        self.registry = {}

        # Create offshore env
        self.oe = plugins.OffshoreEnvironment()
        self.oe.store_information(var='hs', filename='../tests/data/unit_hs.csv')
        self.oe.store_information(var='tp', filename='../tests/data/unit_tp.csv')

    def define_activities(self):
        """Use this function to define your activities."""

        # Define activity
        activity = BasicActivity(
            env=self.env, 
            registry=self.registry,
            name="A basic activity",
            limit_expr=lambda hs, tp: (hs > 0.5) | (tp > 0.5),
            offshore_environment=self.oe,
            duration=3600 * 3
        )

        # Register the activities
        model.register_processes([activity]) 
        
        # Return the base activity for logging purposes
        return [activity]  
        
    def execute_simulation(self):
        """Function starts the simulation."""
        self.activities = self.define_activities()
        self.env.run()
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """Function returns the event log."""
        return get_event_log(activity_list=self.activities)

    @property
    def project_length(self):
        """Function returns the project length."""
        log = self.event_log
        dt = (log.Timestamp.iloc[-1] - log.Timestamp.iloc[0])
        return dt.total_seconds() / (3600)


class BasicActivity(plugins.HasRequestWindowPluginActivity, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [8]:
# Execute simulation
sample_dir = create_sample_data()
sim = SimulationEnvironment(start_date=datetime.datetime(2021, 1, 1))
res = sim.execute_simulation()
sim.event_log

# Remove .csv

os.remove("../tests/data/unit_hs.csv")
os.remove("../tests/data/unit_tp.csv")